### beforehair.py 불러와서 사용자의 현재 얼굴형, 헤어스타일 분석 *--> while 문으로 변경*

##### db에 있는 사용자인지 확인해서 모델 돌리고 결과값 나오게끔 반복문/조건문 만들기

In [ ]:
# 필요 패키지
import os
import json
import requests
import pandas as pd

import beforehairiot

#mqtt통신 프로그램 모듈 추가
import paho.mqtt.client as mqtt

# cpu만 강제 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

user_name = ""
user_phone = ""
user_type = ""
hair_type = ""

def on_message2(client, userdata, msg):
    global user_name,user_phone,user_type, hair_type
    try:
        message1 = str(msg.payload.decode("utf-8"))
        print(message1)
        try:
            message = message1.split(",")
            if(message[0] == 'init_hair'):
                user_name = message[1]
                user_phone = message[2]
                user_type = message[3]
                hair_type = message[4]
        except:
            print("str msg error")
    except:
        try:
            filename = "./test.jpg"
            f = open(filename, "wb")
            f.write(msg.payload)
            print("Image Received")
            f.close()
            image_hair(client,filename)
        except:
            print("file error")

def image_hair(client,imagetest):
    global user_name,user_phone,user_type, hair_type

    try:
        name = user_name
        phone_number = user_phone
        # 얼굴형, 현재 헤어스타일, 헤어 길이 반환
        face_shape, hair_style, hair_length = beforehairiot.face_classify(name,phone_number,imagetest)
        
        # 성별, 컷/펌 임의지정
        gender = user_type
        style = hair_type
        
        if hair_length == '장발':
            hair_length = '1'
        elif hair_length == '중발':
            hair_length = '2'
        elif hair_length == '단발':
            hair_length = '3'
        elif hair_length == '여숏':
            hair_length = '4'
        else:
            hair_length = '5'
        
        # 헤어스타일 추천
        path, hair = beforehairiot.hairstyle_recom(gender, face_shape, style, hair_length)

        # 사용자 정보 json 파일로 반환
        after_table = {'type':'bigdata',
                        'name':name,
                        'face_shape':face_shape,
                        'before_hair':'소프트투블럭댄디',
                        'recom_hair1':path['1'],
                        'recom_hair2':path['2'],
                        'recom_hair3':path['3'],
                        'recom_hair4':path['4'],
                        'after_hair':'보브'}

        print(after_table)

        # 정보 전송
        headers = {'Content-Type': 'application/json; chearset=utf-8'}
        res = requests.post('http://was.t-seonghun.net:8000/insertdata/', data=json.dumps(after_table), headers=headers)
        print(res)
        
        # 헤어스타일 설명
        content = pd.read_csv('./hairstyle_explaination.csv', encoding='cp949')
        content.columns = ['hair_name', 'explain']

        content1 = content[content['hair_name'] == hair['1']]
        content1 = content1['explain'].values
        content1 = "".join(content1)
        
        content2 = content[content['hair_name'] == hair['2']]
        content2 = content2['explain'].values
        content2 = "".join(content2)
        
        content3 = content[content['hair_name'] == hair['3']]
        content3 = content3['explain'].values
        content3 = "".join(content3)
        
        content4 = content[content['hair_name'] == hair['4']]
        content4 = content4['explain'].values
        content4 = "".join(content4)
        
        # mqtt 전달 json
        after_table2 = {'type':'bigdata',
                            'name':name,
                            'face_shape':face_shape,
                            'before_hair':'소프트투블럭댄디',
                            'recom_hair1':path['1'],
                            'content1':content1,
                            'recom_hair2':path['2'],
                            'content2':content2,
                            'recom_hair3':path['3'],
                            'content3':content3,
                            'recom_hair4':path['4'],
                            'content4':content4,
                            'after_hair':'보브'}

        print(after_table2)

        # mqtt 통신으로 json 전달, 서버 닫혀있을 시 에러
        try:
            client.publish('json', json.dumps(after_table2), 1)
        except:
            print("publish error")

    except:
        print("hair image error")
        

#통신을 위한 인스턴스 생성, 서버 닫혀있을 시 에러 발생(클라우드에게 문의)
try:
    client = mqtt.Client()
    client.on_message = on_message2
    client.connect('54.150.133.192', 1883)
    client.subscribe('bigdata/#', 1)
    print("mqtt connect success")
    client.loop_forever()
except:
    print("mqtt server error")

mqtt connect success
init_hair,박성훈,01079233285,man,cut
Image Received
{'type': 'bigdata', 'name': '박성훈', 'face_shape': '둥근형', 'before_hair': '소프트투블럭댄디', 'recom_hair1': './hair_image/recom_hair/man/Round/cut/5/crop3.jpg', 'recom_hair2': './hair_image/recom_hair/man/Round/cut/5/dandy5.JPG', 'recom_hair3': './hair_image/recom_hair/man/Round/cut/5/gail4.jpg', 'recom_hair4': './hair_image/recom_hair/man/Round/cut/5/ivyleague4.jpg', 'after_hair': '보브'}
<Response [500]>
{'type': 'bigdata', 'name': '박성훈', 'face_shape': '둥근형', 'before_hair': '소프트투블럭댄디', 'recom_hair1': './hair_image/recom_hair/man/Round/cut/5/crop3.jpg', 'content1': '이태원클라쓰에서 박서준이 하고 나와 유행하게 된 스타일로, 기장이 짧고, 앞머리가 직선의 형태를 띄는 스타일입니다', 'recom_hair2': './hair_image/recom_hair/man/Round/cut/5/dandy5.JPG', 'content2': '앞머리는 일자느낌이나 비대칭느낌으로 자르며, 윗머리는 무거우면서도 가벼운 라인을 만들어 주는 스타일입니다', 'recom_hair3': './hair_image/recom_hair/man/Round/cut/5/gail4.jpg', 'content3': '아이비리그컷보다 기장이 길며, 앞머리를 반만 올려 이마를 반만 보이게 하는 스타일입니다', 'recom_hair4': './hair_imag